In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
import ktrain

## STEP 1: Get Raw Document Data

In [3]:
# 20newsgroups
from sklearn.datasets import fetch_20newsgroups
remove = ('headers', 'footers', 'quotes')
newsgroups_train = fetch_20newsgroups(subset='train', remove=remove)
newsgroups_test = fetch_20newsgroups(subset='test', remove=remove)
texts = newsgroups_train.data +  newsgroups_test.data

## STEP 2: Train an LDA Topic Model to Discover Topics

The `get_topic_model` function learns a [topic model](https://en.wikipedia.org/wiki/Topic_model) using [Latent Dirichlet Allocation (LDA)](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation).

In [4]:
%%time
tm = ktrain.text.get_topic_model(texts, n_features=10000)

n_topics automatically set to 97
preprocessing texts...
fitting model...
iteration: 1 of max_iter: 5
iteration: 2 of max_iter: 5
iteration: 3 of max_iter: 5
iteration: 4 of max_iter: 5
iteration: 5 of max_iter: 5
done.
CPU times: user 16min 43s, sys: 43min 27s, total: 1h 11s
Wall time: 2min


Compute the topic-probability distribution for each document. We need this to compute document similarity.

In [5]:
%%time
tm.build(texts, threshold=0.25)

done.
CPU times: user 1min 33s, sys: 3min 37s, total: 5min 10s
Wall time: 13.2 s


 We can examine the discovered topics using `print_topics`, `get_topics`, or `topics`.  Here, we will use `print_topics`:

In [6]:
tm.print_topics()

topic 0 | tape adam tim case moved bag quote mass marked zionism
topic 1 | image jpeg images format programs tiff files jfif save lossless
topic 2 | alternative movie film static cycles films philips dynamic hou phi
topic 3 | hell humans poster frank reality kent gerard gant eternal bell
topic 4 | air phd chz kit cbc ups w-s rus w47 mot
topic 5 | dog math great figure poster couldn don trying rushdie fatwa
topic 6 | collaboration nazi fact end expression germany philly world certified moore
topic 7 | gif points scale postscript mirror plane rendering algorithm polygon rayshade
topic 8 | fonts font shell converted iii characters slight composite breaks compress
topic 9 | power station supply options option led light tank plastic wall
topic 10 | transmission rider bmw driver automatic shift gear japanese stick highway
topic 11 | tyre ezekiel ruler hernia appeared appointed supreme man land power
topic 12 | space nasa earth data launch surface solar moon mission planet
topic 13 | israel j

## STEP 3: Select Topics of Interest

Let's combine some technology-related documents into a set of positive examples of technology-focused posts.  We can use these documents as seeds to find new documents about technology. To measure semantic similarity among documents, we will represent each document by its topic probability distribution that we computed above.  We will also compile the document IDs for each document associated with these selected topics.

In [9]:
tech_topics = [51, 85, 94, 22]

## Scoring Documents by Similarity

We will score new documents based on how similar they are to the sample selected above using a [One-Class classifier](https://en.wikipedia.org/wiki/One-class_classification). 

In [10]:
tm.train_scorer(topic_ids=tech_topics)

We can now invoke the `scorer` method to measure the degree to which new documents are similar to our technology-related topics.  Let's use `scorer` to measure the similarity of the remaining documents in the corpus. Note that, although we are applying the scorer to documents within the set corpus used to train the topic model, this is not required. Our `scorer` can be applied to any arbitrary set of documents.

In [11]:
other_topics = [i for i in range(tm.n_topics) if i not in tech_topics]
other_texts = [d['text'] for d in tm.get_docs(topic_ids=other_topics)]

Let's score these documents and place into a Pandas dataframe.

In [12]:
# score documents based on similarity
other_scores = tm.score(other_texts)

In [13]:
# display results in Pandas dataframe
other_preds = [int(score > 0) for score in other_scores]
data = sorted(zip(other_preds, other_scores, other_texts), key=lambda item:item[1], reverse=True)
print('Top Inliers (or Most Similar to Our Technology-Related Topics')
print('\t\tNumber of Predicted Inliners: %s' % sum(other_preds))
df = pd.DataFrame(data, columns=['Prediction', 'Score', 'Text'])
df.head()

Top Inliers (or Most Similar to Our Technology-Related Topics
		Number of Predicted Inliners: 377


,Prediction,Score,Text
0,1,0.212587,I'm looking for recommendations for a laser printer. It will\nbe used mostly for text by a single user. It doesn't need to\nbe a postscript printer. Any advice would be appreciated.\n
1,1,0.211690,"I get the picture, I just find it humorous that Running Windows 3.1 apps ( 3.0 for 2.0 ) \nis what makes os/2 more credible..."
2,1,0.211690,"Two-part question:\n\n1) What is Windows NT - a 'real' windows OS?\n\n2) This past weekend, a local 'hacker' radio show metioned a new product\n from Microsoft called 'Chicago' if I recall. Anyone know what this is?\n\nThat is it -\n\nThanks a heap.\n\n- Alan\n"
3,1,0.211690,Is there any one know:\n\nWhat is the FTP tool for Windows and where to get the tool ?\n\nThanks for any help !!
4,1,0.205488,"Could someone point me toward a source (FTP/BBS/whatever) for development\ntools for the 8051 microprocessor. I specifically am looking for a Macintosh\ncross-assembler/disassembler. Also, is there a mailing-list dedicated to\ndiscussing the 8051? Thanks.\n"


As you can see, we see we've found additional technology-related posts in the dataset.

Our `scorer` assigns a score to each document, where  higher scores indicate a  higher degree of similarity to technology-related seed docments.  The `scorer` implements a decision function to make binary decisions on similarity such that documents with positive scores are deemed as similar and negative scores are deemed dissimilar. We've used this to create  a prediction of 1 for similar and 0 for dissimilar.  This identifies 377 documents as similar.   The `scorer`, however, employs a One-Class classifier, which tends to be more strict.  That is, there are likely documents with negative scores close to zero that are also similar.  Let's look at these.

In [14]:
df[df.Score <=0].head()

,Prediction,Score,Text
377,0,-0.000711,"I was at avalon today and found texture maps in some ""tex"" and ""txc""\nformat, something I've never encountered before. These are obviously\nnot tex or LaTeX files.\n\nIF you have a clue how I can convert these to something\nreasonable, please let me know."
378,0,-0.002478,"I need to be able to cause a beep, but without using any interrupt\nroutines, as I cannot use the BIOS. I believe that the PIC might have\nsomething to do with it, but I'm having troubles deciphering the\ninformation I have on it to figure out how to program it!\n\n\tI'm programming all of this in Turbo C, if that makes any\ndiference at all...\n\n\tPlease can anyone help me??!\n\nThanks,"
379,0,-0.003216,"\nThe only things you'll be able to salvage from the junior are the floppy drives\nand monitor. The floppies are 360k, and the monitor is CGA, but you will need\nan adaptor cable to use it. The junior does not use standard cards. Unless \nyou're really strapped for cash, you should just junk the thing and buy new \nstuff.\n\nDan\n"
380,0,-0.003326,"\nMacintosh II cx with 40 MB HD, 8 MB RAM and 19"" monochrome\nmonitor (Ikegami) is for sale.\nAsking $3,000, no reasonable (best) offer will be rejected.\nContact Konrad at (416) 365-0564m Mon-Frii 9-5.\n"
381,0,-0.003883,"I edited a few newsgroup from that line (don't like to crosspost THAT\nmuch). I can't compare the two, but I recently got an HP DeskJet 500.\n\nI'm very pleased with the output (remember that I'm used to imagens,\nlaser and postscript printers at school -- looks very good. You have\nto be careful to let it dry before touching it, as it will smudge.\n\nThe deskjet is SLOW. This is in comparison to the other printers I\nmentioned. I have no idea how the bubblejet compares.\n\nThe interface between Win3.1 and the printer is just dandy, I've not\nhad any problems with it.\n\nHope that helps some.\n\n--Cindy\n\n--\nCindy Tittle Moore"


As you can see, these documents are also similar and related to technology (albeit slightly different aspects of technology than that of our seed set of documents).  Such negatively-scored documents are useful for identifying so-called informative examples.  Since documents are sorted by score (descending order), we can start at the beginning of the dataframe containing negatively-scored documents and add documents to the positive class until we start seeing negative documents that are **not** related to technology.  These informative negative examples can, then, be added to a negative class for training a traditional binary classsifier.  This process is referred to as [active learning](https://en.wikipedia.org/wiki/Active_learning_(machine_learning)).

For instance, in this example, scores below -0.5 start to become **unrelated** to the themes covered by our technical topics.

In [15]:
df[(df.Score<-0.51)].head()

,Prediction,Score,Text
2217,0,-0.510036,"\nDon't forget Chemical Abstracts Service (which is pretty much the international\nclearinghouse for all chemical information), whose former director (Ronald\nWigington) and head of R&D (Nick Farmer) were openly former NSA employees."
2218,0,-0.510492,"From article <1993Apr21.013846.1374@cx5.com>, by tlc@cx5.com:\n\nAccording to my ColoRIX manual .SCF files are 640x480x256\n\n\nYou may try VPIC, I think it handles the 256 color RIX files OK..\n"
2219,0,-0.510556,What about disks? Won't it erase them if you're carrying them in the bag?
2220,0,-0.510582,"was\nYuppies\nstarted\nYep, that's when I noticed it too. I stopped replacing the hood badge \nafter the second or third one (at $12.00 each).\n\n2002 drivers used to flash their headlight at each other in greeting. Try \nflashing your headlights at a 318i driver and see what kind of look you \nget. They usually check their radar detector...they think you're alerting \nthem to a cop."
2221,0,-0.510692,"refrettably you are mistaken. alt.drugs was used to recruit people for the\nworldwide pot religion. I, however hve no problem being in both of them\n\n"


## Using Keyword Searches to Construct Seed Sets

Let's construct a set of seed documents from a keyword search instead of by LDA-discovered topics.  Let's search all the documents for the word 'Christ':

In [16]:
results = tm.search('Christ', case_sensitive=False)

There are 313 of them.

In [17]:
len(results)

313

Most documents in this set are about Christianity, as expected:

In [18]:
print(results[0]['text'])


Yep, that's pretty much it. I'm not a Jew but I understand that this is the
Jewish way of thinking. However, the Jews believe that the Covenant between
YHWH and the Patriarchs (Abraham and Moses, in this case) establishes a Moral
Code to follow for mankind. Even the Jews could not decide where the boundaries
fall, though.

As I understand it, the Sadducees believed that the Torah was all that was
required, whereas the Pharisees (the ancestors of modern Judaism) believed that
the Torah was available for interpretation to lead to an understanding of
the required Morality in all its nuances (->Talmud).

The essence of all of this is that Biblical Morality is an interface between
Man and YHWH (for a Jew or Christian) and does not necessarily indicate
anything about YHWH outside of that relationship (although one can speculate).


The trouble with all of this is that we don't really know what the "created
in His image" means. I've heard a number of different opinions on this and
have still

Let's construct a positive class from these 313 documents and use them to find other religious documents:

In [19]:
# train scorer from document IDs returned by keyword search
doc_ids = [doc['doc_id'] for doc in results]
tm.train_scorer(doc_ids=doc_ids)

# get text and scores of remaining documents
other_texts = [d['text'] for d in tm.get_docs() if d['doc_id'] not in doc_ids]
other_scores = tm.score(other_texts)

# display results in Pandas dataframe
other_preds = [int(score > 0) for score in other_scores]
data = sorted(zip(other_preds, other_scores, other_texts), key=lambda item:item[1], reverse=True)
print('Top Inliers (or Most Similar to Our Technology-Related Topics')
print('\t\tNumber of Predicted Inliners: %s' % sum(other_preds))
df = pd.DataFrame(data, columns=['Prediction', 'Score', 'Text'])
df.head(3)

Top Inliers (or Most Similar to Our Technology-Related Topics
		Number of Predicted Inliners: 4759


Prediction     Score  \
0  1           0.418117   
1  1           0.409673   
2  1           0.405350   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

Here, we've easily found many other documents about religion-focused documents that **do not** explicitly mention Christ.

Notice that our One Class classifier classified 4759 documents as within-class.  This is because many documents mentioning "Christ" may not be about religion or Christianity.  As a result, our One-Class classifier predicts 1 for non-relgious documents that are also similar to our positive set.  See documents predicted as positive with low scores close to zero:

In [23]:
df[df.Score > 0].tail()

,Prediction,Score,Text
4754,1,0.000214,"Ken:\n\nYour arguments are thoughtful but you are going up against the Big\nBoys if you're tackling Henry. Allen Sherzer will doubtless chime in\non the subject of staggering operational costs, too. Good luck, son.\n\n\nAhem. The Russians are in the Free World now, or at least it would be\nPolitically Correct to contend so.\n\n\nIt will be tough to make DC-X succeed, and to turn it into an\noperational orbital vehicle. Doubtless it will fail to meet some of\nthe promised goals. The reason people are so fond of it is that it's\nthe *only* chance we have now, or will have for a *long* time to come,\nto develop a launch vehicle with radically lower costs. \n\nThere is no Shuttle successor in funded development, NASP is dwindling\naway, and ALS/NLS/Spacelifter sure as hell aren't gonna knock any\nzeroes off that $2000-$3000 per pound cost. Part of the blame for\nthis must be placed on a Shuttle program that consumes many annual\nbillions of the, er, Free World's available space cash. As you will\nno doubt hear from many correspondents in the days to come. (-:\n\nDC-X is an attempt to break out of the vicious cycle by keeping\ndevelopment costs low and flying incremental ""X-plane"" hardware.\nIt's been, to my mind, incredibly successful already-- they've built a\ncomplex prototype in under 600 days for under 60 megabucks. I would\nhave been extremely skeptical that this could be accomplished in 1990s\nAmerica, never mind flying the thing, getting a successor funded, or\nbuilding the DC-Y.\n\nI'm sure you know well that launch costs are THE basic problem for any\nexpansion of astronautics. I don't see a realistic prospect for\nbeating down those costs, for multi-ton payloads, anywhere else. If\nthe DC flops, it'll be business as usual in space. The Nineties and\nthe Double-Oughts will look just like the Seventies and Eighties, a\nprospect too depressing to bear.\n\n(Pegasus represents another assault on the problem from a different\ndirection. It doesn't lower cost-per-pound but it offers an orbital\nlaunch for under ten megabucks. It's creating its own market for\nsmall payloads.)\n\nI read the magazines and I've attended the last two IAFs. There are\nplenty of engineers with paper ideas for cheaper launch systems, some\nof them as good as or better than SSTO. There is no sign in today's\nworld that any of these designs will be allowed anywhere near an\nassembly line.\n\n[...deleting some things I'm not going to prove tonight...]\n\nStrawman. Is anybody seriously proposing this? References, please. \nThe DC must be developed in the real-world funding climate, which\nincludes a NASA ferociously committed to continuing Shuttle\noperations, as well as the ""bird in the hand"" argument your common\nsense tells you. If DC-Y flies at all, it flies alongside the\nShuttle, not instead of it.\n\nAlso, of course, DC-Y and its operational descendants will be useful\nfor a wide variety of jobs even if they are *not* man-rated.\n\n\nIf a DC-X successor can fly a 10,000-kg payload for $1M, or even $5M,\nrather than the $40M it now costs, more people will be able to afford\nmore payloads... for the same money, you can fly several satellites\ninstead of one. Big outfits can fly multi-satellite series. Little\noutfits will be able to fly spacecraft of their own, instead of\nbegging a ride. This is just supply and demand. You should be able\nto convince *yourself* that point 4 will be true, assuming DC makes a\nbig difference in costs. Do you have some reason to think not?"
4755,1,0.000159,"Wanted: Summer sublet in NW DC, on red Metro line. Have own bedroom, but can\n share common areas with others. Apartment or room for $400 or less.\n Move in Memorial Day weekend through end of August. No smokers."
4756,1,0.000159,"Wanted: Summer sublet in NW DC, on red Metro line. Have own bedroom, but can\n share common areas with others. Apartment or room for $400 or less.\n Move in Memorial Day weekend through end of Au